In [ ]:
from time import time
import bitstring
import string
import random as rand

# initial, inverse, and E-bit permutation arrays
Init_Perm = [2, 6, 3, 1, 4, 8, 5, 7]
Inv_Perm = [4, 1, 3, 5, 7, 2, 8, 6]
E_P = [4, 1, 2, 3, 2, 3, 4, 1]

# Permutation arrays
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]
P4 = [2, 4, 3, 1]

#S boxes for substitutions
S = {
  "00": 0,
  "01": 1,
  "10": 2,
  "11": 3
}
S1 = [["01", "00", "11", "10"],
      ["11", "10", "01", "00"],
      ["00", "10", "01", "11"],
      ["11", "01", "11", "10"]]
S2 = [["00", "01", "10", "11"],
      ["10", "00", "01", "11"],
      ["11", "00", "01", "00"],
      ["10", "01", "00", "11"]]

def Subs(s0, s1):
    w = S[s0[0] + s0[3]]
    x = S[s0[1] + s0[2]]
    y = S[s1[0] + s1[3]]
    z = S[s1[1] + s1[2]]
    return S1[w][x] + S2[y][z]

# uses the length of the key divided by 2 to return the Left or right of the permuted key for C0 and D0 respectively
def Left(permedKey):
    return permedKey[:int((len(permedKey)/2))]

def Right(permedKey):
    return permedKey[int((len(permedKey)/2)):]

# Leftshift for key generation
def LeftShift(unshifted, shifts):
    if shifts == 1:
        return unshifted[1:] + unshifted[0]
    elif shifts == 2:
        return unshifted[2:] + unshifted[0] + unshifted[1]

#Permutation for SDES
def Perm(oldString, perm):
    newString = ''
    for i in perm:
        newString += oldString[i-1]
    return newString

def XOR_Part(bits, key):
    new = ''
    for bit, key_bit in zip(bits, key):
        new += str(((int(bit) + int(key_bit)) % 2))
    return new
    
# SDES Function
def SDES(plaintext,Key):

    #----------------------------------- Key Generation Steps ----------------------------------------------

    ChangedKey = ''

    # 10 bit key permutation
    ChangedKey = Perm(Key, P10)
    
    # splitting key into C_0 and D_0
    C_0 = Left(ChangedKey)
    D_0 = Right(ChangedKey)
    
    #First left shifts (1 left shift)
    C_1 = LeftShift(C_0, 1)
    D_1 = LeftShift(D_0, 1)
    
    # First Round Key
    Key1 = C_1 + D_1
    RK_1 = Perm(Key1, P8)

    # Second Left shift (2 left shifts)
    C_2 = LeftShift(C_1, 2)
    D_2 = LeftShift(D_1, 2)

    # Second Round Key
    Key2 = C_2 + D_2
    RK_2 = Perm(Key2, P8)

    # Third Left shift (2 left shifts)
    C_3 = LeftShift(C_2, 2)
    D_3 = LeftShift(D_2, 2)

    # Third Round Key
    Key3 = C_3 + D_3
    RK_3 = Perm(Key3, P8)

    # Fourth Left shift (2 left shifts)
    C_4 = LeftShift(C_3, 2)
    D_4 = LeftShift(D_3, 2)

    # Fourth Round Key
    Key4 = C_4 + D_4
    RK_4 = Perm(Key4, P8)
    
    # ------------------------------------ Actual SDES Rounds --------------------------------------------
    
    #Initial Permutation
    Alt_Mess = Perm(plaintext, Init_Perm)
    
    #split into sides
    L_0 = Left(Alt_Mess)
    R_0 = Right(Alt_Mess)
    
    # E-bit Selection
    Alt_R0 = Perm(R_0, E_P)
    
    #XOR step
    XOR_output = XOR_Part(Alt_R0, RK_1)
    
    #Substitution
    s0 = Left(XOR_output)
    s1 = Right(XOR_output)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again = XOR_Part(L_0, p4)
    
    # MESSAGE AFTER FIRST ROUND OF SDES
    Alt_Mess1 = R_0 + XOR_again
    
    #split into sides
    L_1 = R_0
    R_1 = XOR_again
    
    # E-bit Selection
    Alt_R1 = Perm(R_1, E_P)
    
    #XOR step
    XOR_output1 = XOR_Part(Alt_R1, RK_2)
    
    #Substitution
    s0 = Left(XOR_output)
    s1 = Right(XOR_output)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again1 = XOR_Part(L_1, p4)
    
    # MESSAGE AFTER SECOND ROUND OF SDES
    Alt_Mess2 = R_1 + XOR_again1
    
    #split into sides
    L_2 = R_1
    R_2 = XOR_again1
    
    # E-bit Selection
    Alt_R2 = Perm(R_2, E_P)
    
    #XOR step
    XOR_output2 = XOR_Part(Alt_R2, RK_3)
    
    #Substitution
    s0 = Left(XOR_output2)
    s1 = Right(XOR_output2)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again2 = XOR_Part(L_2, p4)
    
    # MESSAGE AFTER THIRD ROUND OF SDES
    Alt_Mess3 = R_2 + XOR_again2
    
    #split into sides
    L_3 = R_2
    R_3 = XOR_again2
    
    # E-bit Selection
    Alt_R3 = Perm(R_3, E_P)
    
    #XOR step
    XOR_output3 = XOR_Part(Alt_R3, RK_4)
    
    #Substitution
    s0 = Left(XOR_output3)
    s1 = Right(XOR_output3)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again3 = XOR_Part(L_3, p4)
    
    # MESSAGE AFTER FOURTH ROUND OF SDES
    Alt_Mess4 = R_3 + XOR_again3
    
    #split into sides
    L_4 = R_3
    R_4 = XOR_again3
    PreOutput = R_4 + L_4
    
    #Final Encypted plaintext
    ciphertext = Perm(PreOutput, Inv_Perm)
    return ciphertext


#-------------------------------------------Key Gen Part -----------------------------------------------------------
def Key_Gen(Key):
    ChangedKey = ''

    # 10 bit key permutation
    ChangedKey = Perm(Key, P10)
    
    # splitting key into C_0 and D_0
    C_0 = Left(ChangedKey)
    D_0 = Right(ChangedKey)
    
    #First left shifts (1 left shift)
    C_1 = LeftShift(C_0, 1)
    D_1 = LeftShift(D_0, 1)
    
    # First Round Key
    Key1 = C_1 + D_1
    RK_1 = Perm(Key1, P8)

    # Second Left shift (2 left shifts)
    C_2 = LeftShift(C_1, 2)
    D_2 = LeftShift(D_1, 2)

    # Second Round Key
    Key2 = C_2 + D_2
    RK_2 = Perm(Key2, P8)

    # Third Left shift (2 left shifts)
    C_3 = LeftShift(C_2, 2)
    D_3 = LeftShift(D_2, 2)

    # Third Round Key
    Key3 = C_3 + D_3
    RK_3 = Perm(Key3, P8)

    # Fourth Left shift (2 left shifts)
    C_4 = LeftShift(C_3, 2)
    D_4 = LeftShift(D_3, 2)

    # Fourth Round Key
    Key4 = C_4 + D_4
    RK_4 = Perm(Key4, P8)
    
    arr_RK = [RK_1, RK_2, RK_3, RK_4]
    return arr_RK

#----------------------------------------------------Info for Key search-----------------------------------------------------

key_bit = string.printable[:2]
keyLength = 20

def Potential_Key(keyLength):
    Key=''
    while len(Key) < keyLength:
        Key += rand.choice(key_bit)
    return Key
Poss_Key = Potential_Key(keyLength)
    
    # This was to find and print the weak keys
    #for i in  range(len(all_keys)):
        #arr = Key_Gen(all_keys[i])
        #if arr[0] == arr[1] and arr[1] == arr[2] and arr[2] == arr[3]:
            #print(all_keys[i])
# --------------------------------------------------Double SDES Function--------------------------------------------------

def D_SDES(Key1, Key2, plaintext):
    plaintext2 = SDES(plaintext,Key1)
    ciphertext = SDES(plaintext2, Key2)
    return ciphertext
#--------------------------------------------------SDES Decrypt----------------------------------------------------------
def Dec_SDES(ciphertext, Key):

    #----------------------------------- Key Generation Steps ----------------------------------------------

    ChangedKey = ''

    # 10 bit key permutation
    ChangedKey = Perm(Key, P10)
    
    # splitting key into C_0 and D_0
    C_0 = Left(ChangedKey)
    D_0 = Right(ChangedKey)
    
    #First left shifts (1 left shift)
    C_1 = LeftShift(C_0, 1)
    D_1 = LeftShift(D_0, 1)
    
    # First Round Key
    Key1 = C_1 + D_1
    RK_1 = Perm(Key1, P8)

    # Second Left shift (2 left shifts)
    C_2 = LeftShift(C_1, 2)
    D_2 = LeftShift(D_1, 2)

    # Second Round Key
    Key2 = C_2 + D_2
    RK_2 = Perm(Key2, P8)

    # Third Left shift (2 left shifts)
    C_3 = LeftShift(C_2, 2)
    D_3 = LeftShift(D_2, 2)

    # Third Round Key
    Key3 = C_3 + D_3
    RK_3 = Perm(Key3, P8)

    # Fourth Left shift (2 left shifts)
    C_4 = LeftShift(C_3, 2)
    D_4 = LeftShift(D_3, 2)

    # Fourth Round Key
    Key4 = C_4 + D_4
    RK_4 = Perm(Key4, P8)
    
    # ------------------------------------ SDES Rounds --------------------------------------------
    
    #Initial Permutation
    Alt_Mess = Perm(ciphertext, Init_Perm)
    
    #split into sides
    L_0 = Left(Alt_Mess)
    R_0 = Right(Alt_Mess)
    
    # E-bit Selection
    Alt_R0 = Perm(R_0, E_P)
    
    #XOR step
    XOR_output = XOR_Part(Alt_R0, RK_4)
    
    #Substitution
    s0 = Left(XOR_output)
    s1 = Right(XOR_output)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again = XOR_Part(L_0, p4)
    
    # MESSAGE AFTER FIRST ROUND OF SDES
    Alt_Mess1 = R_0 + XOR_again
    
    #split into sides
    L_1 = R_0
    R_1 = XOR_again
    
    # E-bit Selection
    Alt_R1 = Perm(R_1, E_P)
    
    #XOR step
    XOR_output1 = XOR_Part(Alt_R1, RK_3)
    
    #Substitution
    s0 = Left(XOR_output)
    s1 = Right(XOR_output)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again1 = XOR_Part(L_1, p4)
    
    # MESSAGE AFTER SECOND ROUND OF SDES
    Alt_Mess2 = R_1 + XOR_again1
    
    #split into sides
    L_2 = R_1
    R_2 = XOR_again1
    
    # E-bit Selection
    Alt_R2 = Perm(R_2, E_P)
    
    #XOR step
    XOR_output2 = XOR_Part(Alt_R2, RK_2)
    
    #Substitution
    s0 = Left(XOR_output2)
    s1 = Right(XOR_output2)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again2 = XOR_Part(L_2, p4)
    
    # MESSAGE AFTER THIRD ROUND OF SDES
    Alt_Mess3 = R_2 + XOR_again2
    
    #split into sides
    L_3 = R_2
    R_3 = XOR_again2
    
    # E-bit Selection
    Alt_R3 = Perm(R_3, E_P)
    
    #XOR step
    XOR_output3 = XOR_Part(Alt_R3, RK_1)
    
    #Substitution
    s0 = Left(XOR_output3)
    s1 = Right(XOR_output3)
    s = Subs(s0, s1)
    
    #permutation into 4 bits again
    p4 = Perm(s, P4)
    XOR_again3 = XOR_Part(L_3, p4)
    
    # MESSAGE AFTER FOURTH ROUND OF SDES
    Alt_Mess4 = R_3 + XOR_again3
    
    #split into sides
    L_4 = R_3
    R_4 = XOR_again3
    PreOutput = R_4 + L_4
    
    #Final Encypted plaintext
    plaintext = Perm(PreOutput, Inv_Perm)
    return plaintext

# -------------------------------------Meet in the Middle Breaking-----------------------------------------------------------
def MITM(pt1, ct1, pt2, ct2, pt3, ct3, pt4, ct4, pt5, ct5):
    for i in range(len(all_keys)):
        for j in range(len(all_keys)):
            c1 = SDES(pt1, all_keys[i])
            c1_2 = Dec_SDES(ct1, all_keys[j])
            c2 = SDES(pt2, all_keys[i])
            c2_2 = Dec_SDES(ct2, all_keys[j])
            c3 = SDES(pt3, all_keys[i])
            c3_2 = Dec_SDES(ct3, all_keys[j])
            c4 = SDES(pt4, all_keys[i])
            c4_2 = Dec_SDES(ct4, all_keys[j])
            c5 = SDES(pt5, all_keys[i])
            c5_2 = Dec_SDES(ct5, all_keys[j])
            if c1 == c1_2 and c2 == c2_2 and c3 == c3_2 and c4 == c4_2 and c5 == c5_2:
                print(all_keys[i] + all_keys[j])


#-------------------------------------------------Brute Force Breaking------------------------------------------------------
            
def Brute(pt1, ct1, pt2, ct2, pt3, ct3, pt4, ct4, pt5, ct5):
    arr1 = []
    arr2 = []
    arr3 = []
    arr4 = []
    arr5 = []
    for i in range(len(all_keys)):
        for j in range(len(all_keys)):
            c1 = D_SDES(all_keys[i], all_keys[j], pt1)
            c2 = D_SDES(all_keys[i], all_keys[j], pt2)
            c3 = D_SDES(all_keys[i], all_keys[j], pt3)
            c4 = D_SDES(all_keys[i], all_keys[j], pt4)
            c5 = D_SDES(all_keys[i], all_keys[j], pt5)
            if c1 == ct1:
                arr1.append(all_keys[i] + all_keys[j])
            elif c2 == ct2:
                arr2.append(all_keys[i] + all_keys[j])
            elif c3 == ct3:
                arr3.append(all_keys[i] + all_keys[j])
            elif c4 == ct4:
                arr4.append(all_keys[i] + all_keys[j])
            elif c5 == ct5:
                arr5.append(all_keys[i] + all_keys[j])
    for i in range(len(arr1)):
        if arr1[i] in arr2 and arr1[i] in arr3 and arr1[i] in arr4 and arr1[i] in arr5:
            return arr1[i]  

if __name__ == "__main__": 
    all_keys = []
    for a in range(len(key_bit)):
        for b in range(len(key_bit)):
            for c in range(len(key_bit)):
                for d in range(len(key_bit)):
                    for e in range(len(key_bit)):
                        for f in range(len(key_bit)):
                            for g in range(len(key_bit)):
                                for h in range(len(key_bit)):
                                    for i in range(len(key_bit)):
                                        for j in range(len(key_bit)):
                                            KEY = key_bit[a] + key_bit[b] + key_bit[c] + key_bit[d] + key_bit[e] + key_bit[f] + key_bit[g] + key_bit[h] + key_bit[i] + key_bit[j]
                                            all_keys.append(KEY)
                                            
    start = time()
    #Brute('01000010', '01010010', '01110010', '11110000', '01110101', '10111110', '01110100', '01101001', '01100101', '10001010')
    MITM('01000010', '01010010', '01110010', '11110000', '01110101', '10111110', '01110100', '01101001', '01100101', '10001010')
    end = time()
    Time = end - start
    print(Time)
    

457.4509084224701
